In [1]:
import numpy as np
import sys
import scipy.stats as st
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations
sys.path.append('../multiKS/')
from multiKS import multiKS
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
dim = 2
mu = [0]*dim
sigma = np.identity(dim)
rv = st.multivariate_normal(mu, sigma)

In [18]:
mc_loops_per_core = 40
n_cores = 8
N = 50
samples = [[rv.rvs(N) for i in range(mc_loops_per_core)] for j in range(n_cores)]

def KS_wrapper(Xs):
    return [multiKS(X, rv.cdf) for X in Xs]

In [19]:
with ProcessPoolExecutor() as executor:
    results = executor.map(KS_wrapper, samples)

In [23]:
np.save(f'KS_results_dim={dim}.npy', list(results))